In [1]:
import requests
import scaleapi
import json
from os import environ


Retrieving Project

In [ ]:
import requests

url = "https://api.scale.com/v1/projects/Traffic Sign Detection"

headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", url, headers=headers)
pretty_json = json.loads(response.text)
print (json.dumps(pretty_json, indent=4))


Retrieving Tasks

In [ ]:
import requests

task_retrieve_url = "https://api.scale.com/v1/tasks?status=completed&project=Traffic Sign Detection&limit=100"
task_detail_url = "https://api.scale.com/v1/task/"
accepted_annotation_label = ["traffic_control_sign",
                                "construction_sign",
                                "information_sign",
                                "policy_sign",
                                "non_visible_face"
                            ]


headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", task_retrieve_url, headers=headers)

pretty_response = json.loads(response.text)

# list of tasks
tasks = pretty_response["docs"] 

for i in tasks:
    print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", task_detail_url+i["task_id"], headers=headers)

    pretty_response_task = json.loads(response_task.text)

    print (json.dumps(pretty_response_task, indent=4))

    # responses = pretty_response_task["response"]["annotations"]
    
# 


# print (json.dumps(pretty_json["docs"][7]["task_id"], indent=4))